In [91]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

In [92]:
#read excel, skip to row 10 and column 39
df = pd.read_excel('all-countries-2017-2023.xlsx', skiprows=11, usecols='A:W')

In [93]:
df.columns

Index(['Country', 'Level 1 Name', 'Area', 'Area ID', 'Analysis Name',
       'Date of Analysis', 'Country Population', '#', '% of total county Pop',
       'Area Phase', 'Analysis Period', '#.1', '%', '#.2', '%.1', '#.3', '%.2',
       '#.4', '%.3', '#.5', '%.4', '#.6', '%.5'],
      dtype='object')

In [94]:
#rename column '#.1', '%', '#.2', '%.1',
df.rename(columns={ 'Country Population': 'population',
                   '#': 'population_analyzed', '% of total county Pop': 'pop_per',
                   '#.1':'phase1_abs', '%':'phase1_per', 
                   '#.2':'#phase2_abs', '%.1':'phase2_abs',
                    '#.3':'phase3_abs', '%.2':'phase3_per',
                    '#.4':'phase4_abs', '%.3':'phase4_per',
                    '#.5':'phase5_abs', '%.4':'phase5_per',
                    '#.6':'phase3plus_abs', '%.5':'phase3plus_per',
                    'Date of Analysis':'date'}, inplace=True)


In [95]:
df.shape

(11100, 23)

In [96]:
#remove rows with NaN values in the 'Country' column
df = df.dropna(subset=['Country'])

#make a seperate df full country data
df_countries = df[df['Area'].isna()]

#convert date to datetime
df_countries['date'] = pd.to_datetime(df_countries['date'])

/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_6656/2683068199.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_countries['date'] = pd.to_datetime(df_countries['date'])
/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_6656/2683068199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['date'] = pd.to_datetime(df_countries['date'])


In [97]:
#keep only rows where country contains the term "acute food insecurity"
df_countries = df_countries[df_countries['Country'].str.contains('Acute Food Insecurity')]
df_countries['Country'] = df_countries['Country'].str.split(':').str[0]


In [99]:
#keep only Country, population, phase3plus_abs, phase3plus_per, date
df_countries = df_countries[['Country', 'population', 'phase3plus_abs', 'phase3plus_per', 'date']]
df_countries

#save as csv
df_countries.to_csv('acutefoodinsecurity.csv', index=False)

In [45]:
# Read csv
fs='https://hapi.humdata.org/api/v1/food/food-security?ipc_phase=3%2B&ipc_type=current&location_code=YEM&output_format=json&app_identifier=Y2dlbGlsX2xzZTpjLmFiZGVsZ2VsaWxAbHNlLmFjLnVr&limit=10000&offset=0'
fs_df = pd.json_normalize(requests.get(fs).json()['data'])



In [47]:
fs_df

,location_ref,location_code,location_name,admin1_ref,admin1_code,admin1_name,provider_admin1_name,admin2_ref,admin2_code,admin2_name,provider_admin2_name,resource_hdx_id,ipc_phase,ipc_type,population_in_phase,population_fraction_in_phase,reference_period_start,reference_period_end
0,245,YEM,Yemen,2478,YE12,Abyan,Abyan,30100,YE1201,Al Mahfad,Al mahfad,fe7aef6a-df83-4e5d-a625-75cfaf287bd2,3+,current,19542,0.50,2024-07-01T00:00:00,2024-09-30T00:00:00
1,245,YEM,Yemen,2478,YE12,Abyan,Abyan,30101,YE1202,Mudiyah,Mudiyah,fe7aef6a-df83-4e5d-a625-75cfaf287bd2,3+,current,25540,0.50,2024-07-01T00:00:00,2024-09-30T00:00:00
2,245,YEM,Yemen,2478,YE12,Abyan,Abyan,30102,YE1203,Jayshan,Jayshan,fe7aef6a-df83-4e5d-a625-75cfaf287bd2,3+,current,12108,0.55,2024-07-01T00:00:00,2024-09-30T00:00:00
3,245,YEM,Yemen,2478,YE12,Abyan,Abyan,30103,YE1204,Lawdar,Lawdar,fe7aef6a-df83-4e5d-a625-75cfaf287bd2,3+,current,53403,0.50,2024-07-01T00:00:00,2024-09-30T00:00:00
4,245,YEM,Yemen,2478,YE12,Abyan,Abyan,30104,YE1205,Sibah,Sibah,fe7aef6a-df83-4e5d-a625-75cfaf287bd2,3+,current,10217,0.40,2024-07-01T00:00:00,2024-09-30T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,245,YEM,Yemen,2492,YE26,Ma'rib,Marib,33165,None,None,,741a6164-c8b8-413b-b021-53d250a5814b,3+,current,876464,0.54,2024-07-01T00:00:00,2024-09-30T00:00:00
127,245,YEM,Yemen,2494,YE28,Al Maharah,Al Maharah,33167,None,None,,741a6164-c8b8-413b-b021-53d250a5814b,3+,current,71119,0.36,2024-07-01T00:00:00,2024-09-30T00:00:00
128,245,YEM,Yemen,2496,YE30,Ad Dali',Al Dhale'e,33169,None,None,,741a6164-c8b8-413b-b021-53d250a5814b,3+,current,239578,0.47,2024-07-01T00:00:00,2024-09-30T00:00:00
129,245,YEM,Yemen,2498,YE32,Socotra,Socotra,33171,None,None,,741a6164-c8b8-413b-b021-53d250a5814b,3+,current,29574,0.25,2024-07-01T00:00:00,2024-09-30T00:00:00
